In [5]:
# Libraries
import os
import time
import json
import math
import requests
import logging
import pandas as pd
import numpy as np

In [19]:
# Urls for fetching Data
url_oc      = "https://www.nseindia.com/option-chain"
url_nf      = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'

# Headers
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
           'accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'en-GB,en;q=0.9',
            'accept-encoding': 'gzip, deflate, br, zstd'}

sess = requests.Session()
cookies = dict()

In [29]:
resp = requests.get(url_oc, headers=headers)

In [25]:
request = sess.get(url_oc, headers=headers, timeout=100)

In [26]:
request.status_code

200

In [30]:
print(resp.text)

��#���CDT�z 4R��!��Wo[���W�!�d�q��{\
�43��ͅ$M�$mqR�~I�t������ib�IO��kdL��D\^D[�-s��.�:��{²,��s$Ѱ�$��Su�R$C�R� 3���-�6�g�"M���R|�.�f��RN��c��v8�0��%Q	ʂT�R�mJ��
W�4pkl� RkHV>-�ai��KKnb�;��/�:�k�k_Ԥ~\��� \���~��C���.��U,���m{/_�L����	���J����4�(��Q��r
u�����[�m�߷(R��ΘPhQ��M�B�1��NZ�(EJ��>��4{�)��G���ʨ��`��BN�]3���)�#9��^���<e#�Fb�4���7�-9���:�C����Rf��(<;�dv�ig2 �f9.r��w)�7���iH@�<�:�en�ѱkY��hPJi��O�_դޯ(G -D�TM��P�	�"�=�H��ǵ�Ķ�$�\j̲-ce8��٦a[��Y	[%�J�F{�fD��S#g�r1r�oi�����V�E����FJ����2��pP���߆�J�vѴ��,�?P�
�9*��`�Z����,���hgI�&����:Z*G ���־Â���e`�B��A�G����m5-�����dЬ|{E���>\���H\'�3���.K�E�QQ��t⡓��%p�����y̫��Y��*2�(���q-lX�Ew�:�&�T<������gk��)菦�>��~�Ҧm�}�8��F<;B����X���N���T�$-p��EѢ��V*�/XE�Pn��,\��4�� ���Dǝ^o������aG�TMkѮ���	o^�b�9PF�7>7M�3���8_�X� I	�������������NwJY�E���f���{:���U�o���7!��a��a�n�������^�)��4fqIMY"7'l�0��ȓt��6`%��0���=��f|,%ɳJ�<y`��I��M|� <l��H��^$�H$�&

In [3]:
# Local methods
def set_cookie():
    request = sess.get(url_oc, headers=headers, timeout=5)
    cookies = dict(request.cookies)
    
def get_data(url):
    set_cookie()
    response = sess.get(url, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==401):
        set_cookie()
        response = sess.get(url_nf, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==200):
        return response.text
    return ""

In [11]:
import requests
import json
import pandas as pd

In [15]:
import requests
import json
import pandas as pd
import time # Import time for delays

def get_nse_option_chain(symbol="NIFTY", expiry_date=None):
    initial_url = "https://www.nseindia.com/option-chain"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Referer': 'https://www.nseindia.com/option-chain'
    }

    session = requests.Session()

    try:
        # First request to get initial cookies
        print(f"Attempting to get initial cookies from: {initial_url}")
        initial_response = session.get(initial_url, headers=headers)
        initial_response.raise_for_status() # Check for HTTP errors
        print(f"Initial response status: {initial_response.status_code}")
        cookies = session.cookies.get_dict()
        print(f"Cookies obtained: {cookies}")
        time.sleep(1) # Small delay

        # Construct the API URL
        api_url = f"https://www.nseindia.com/api/option-chain-indices?symbol={symbol.upper()}"
        print(f"Attempting to fetch data from API: {api_url}")

        response = session.get(api_url, headers=headers, cookies=cookies)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)

        print(f"API response status: {response.status_code}")
        print(f"API response content (first 500 chars): \n{response.text[:500]}") # Print part of the content

        # Now try to parse JSON
        data = response.json()

        # ... (rest of your data processing logic)
        option_chain_data = []
        records = data.get('records', {})

        # If a specific expiry_date is provided, filter for it
        if expiry_date:
            filtered_data = [
                item for item in records.get('data', [])
                if item.get('expiryDate') == expiry_date
            ]
        else:
            filtered_data = records.get('data', [])

        for item in filtered_data:
            strike_price = item.get('strikePrice')
            expiry_date = item.get('expiryDate')

            # Extract CE data
            ce_data = item.get('CE', {})
            ce_oi = ce_data.get('openInterest')
            ce_chng_in_oi = ce_data.get('changeinOpenInterest')
            ce_volume = ce_data.get('totalTradedVolume')
            ce_iv = ce_data.get('impliedVolatility')
            ce_ltp = ce_data.get('lastPrice')
            ce_net_change = ce_data.get('change')
            ce_bid_qty = ce_data.get('bidQty')
            ce_bid_price = ce_data.get('bidprice')
            ce_ask_price = ce_data.get('askPrice')
            ce_ask_qty = ce_data.get('askQty')

            # Extract PE data
            pe_data = item.get('PE', {})
            pe_oi = pe_data.get('openInterest')
            pe_chng_in_oi = pe_data.get('changeinOpenInterest')
            pe_volume = pe_data.get('totalTradedVolume')
            pe_iv = pe_data.get('impliedVolatility')
            pe_ltp = pe_data.get('lastPrice')
            pe_net_change = pe_data.get('change')
            pe_bid_qty = pe_data.get('bidQty')
            pe_bid_price = pe_data.get('bidprice')
            pe_ask_price = pe_data.get('askPrice')
            pe_ask_qty = pe_data.get('askQty')

            option_chain_data.append({
                'Strike Price': strike_price,
                'Expiry Date': expiry_date,
                'CE - Open Interest': ce_oi,
                'CE - Change in OI': ce_chng_in_oi,
                'CE - Volume': ce_volume,
                'CE - IV': ce_iv,
                'CE - LTP': ce_ltp,
                'CE - Net Change': ce_net_change,
                'CE - Bid Qty': ce_bid_qty,
                'CE - Bid Price': ce_bid_price,
                'CE - Ask Price': ce_ask_price,
                'CE - Ask Qty': ce_ask_qty,
                'PE - Open Interest': pe_oi,
                'PE - Change in OI': pe_chng_in_oi,
                'PE - Volume': pe_volume,
                'PE - IV': pe_iv,
                'PE - LTP': pe_ltp,
                'PE - Net Change': pe_net_change,
                'PE - Bid Qty': pe_bid_qty,
                'PE - Bid Price': pe_bid_price,
                'PE - Ask Price': pe_ask_price,
                'PE - Ask Qty': pe_ask_qty,
            })

        df = pd.DataFrame(option_chain_data)
        return df

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code} - {e.response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Network or other Request Error: {e}")
        return None
    except json.JSONDecodeError:
        print("Error decoding JSON. The response might not be valid JSON.")
        print(f"Full response content: \n{response.text}") # Print full content for debugging
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example Usage:
df_nifty = get_nse_option_chain(symbol="NIFTY")
if df_nifty is not None:
    print("NIFTY Option Chain:")
    print(df_nifty.head())

Attempting to get initial cookies from: https://www.nseindia.com/option-chain
Initial response status: 200
Cookies obtained: {'_abck': 'FBE25FD52E07F61B56B236FA6162C012~-1~YAAQRLoTAsOZFMqWAQAAYa3ZAw0dIrM1AR3K5hHT5MtiMJI/KRV3Iy7LGYwaE5ysOioBCibC2Xbap2yIqP4gEx/45+Ol1vHFx1UFraUx78CKYX7Fkhymvy7g062Xb7lNqB3J/H/s67ZdvoexNrmnt9Mvf3J0ETqscTR0vfjGCzE+elC2O7bpCS/XxdUV4FgMpnN5Pv/tDNItX0pGV8IzsruUG2iW/xWYSRzsTwSj/2HsFFiP41E6OsCGmBz3AX+Mq+62kbNFI11oy+0pw3+UIEiuwCSRV1FEqMsefOBHVfgpM0dkHAnm1lhlWsonTo4/A0/I71vkqTcRZFQF/wFk6xNfd545sD4/qJwAXSRy1uS35nztD4uoDTQcb6YDSrAVShEzpx7VnnL+GjGkvzghR36iobACcilyuTi5UqM17Yn8fnSmNlR05esmjuOYGEtwqjklEtRu~-1~-1~-1', 'bm_sz': '66089CEFDD6106722B1F649429DCD159~YAAQRLoTAsSZFMqWAQAAYa3ZAxtGjRwMzii2FWjW56lclSD+0yWr1qiiEOwC6nVHaR/4f40Y7bz2v0BgFv6aBlIwE1qCMR/SQeCw0UJmNmoc2PVMq+A5DNqidIurBRS3u5wrlDaJnE+0HtQ5nq/W0wPAPP0oQDUGQGKbn2mWXzwTVllLAk0sHsHtKgNNERjMHhZsQJsd4r4sKRzYkT4ZpiL1pps8i7j3Ho5do19QFlL4DSVO4zDM5wGOkoynBHxh9G/BHt95M4PJk2E3+wmacBvKoSgG7aeEC6RsKKzeTAr/epfyQX5FVYIZra2uU

In [13]:
get_nse_option_chain(symbol="NIFTY", expiry_date="29-May-2025")

Error fetching data: Expecting value: line 1 column 1 (char 0)


In [ ]:
expiry_date=None